In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("github_token")
import shutil

# shutil.rmtree('/kaggle/working/Gender-Classifier')

!git clone https://{secret_value_0}@github.com/Frontman-11/Gender-Classifier.git /kaggle/working/Gender-Classifier

Cloning into '/kaggle/working/Gender-Classifier'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 223 (delta 39), reused 0 (delta 0), pack-reused 148 (from 1)
Receiving objects: 100% (223/223), 25.50 MiB | 11.50 MiB/s, done.
Resolving deltas: 100% (124/124), done.


# SUMMARY

${This\ notebook\ focus\ on\ the\ training\ and\ saving\ of\ the\ model.}$

In [2]:
import sys
sys.path.append('/kaggle/working/Gender-Classifier/')
import preprocess
import train
from model import create_model, init_weights
from utils import stack_filepaths

import tensorflow as tf
import glob

import importlib
# import image_utils
# importlib.reload(image_utils)

2024-08-18 16:20:14.537987: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 16:20:14.538099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 16:20:14.672889: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Seeding

In [3]:
tf.random.set_seed(42)

## Datasets

### ${Gathering\ Datasets\ Filepaths}$

In [4]:
train_male_pattern = '/kaggle/input/male-tfrecord-dataset/train_male.tfrecord-*-of-*'
train_female_pattern = '/kaggle/input/female-tfrecord-dataset/train_female.tfrecord-*-of-*'

valid_male_pattern = '/kaggle/input/tfrecord-writing/valid_male.tfrecord-*-of-*'
valid_female_pattern = '/kaggle/input/tfrecord-writing/valid_female.tfrecord-*-of-*'


train_male_filepaths = glob.glob(train_male_pattern)
train_female_filepaths = glob.glob(train_female_pattern)

valid_male_filepaths = glob.glob(valid_male_pattern)
valid_female_filepaths = glob.glob(valid_female_pattern)

train_filepaths = stack_filepaths([train_male_filepaths, train_female_filepaths])
valiid_filepaths = valid_male_filepaths + valid_female_filepaths

train_file_dataset = tf.data.Dataset.from_tensor_slices(train_filepaths)
valid_file_dataset = tf.data.Dataset.from_tensor_slices(valiid_filepaths)

list(train_filepaths)

['/kaggle/input/male-tfrecord-dataset/train_male.tfrecord-18-of-20',
 '/kaggle/input/female-tfrecord-dataset/train_female.tfrecord-16-of-20',
 '/kaggle/input/male-tfrecord-dataset/train_male.tfrecord-19-of-20',
 '/kaggle/input/female-tfrecord-dataset/train_female.tfrecord-19-of-20',
 '/kaggle/input/male-tfrecord-dataset/train_male.tfrecord-03-of-20',
 '/kaggle/input/female-tfrecord-dataset/train_female.tfrecord-20-of-20',
 '/kaggle/input/male-tfrecord-dataset/train_male.tfrecord-13-of-20',
 '/kaggle/input/female-tfrecord-dataset/train_female.tfrecord-13-of-20',
 '/kaggle/input/male-tfrecord-dataset/train_male.tfrecord-06-of-20',
 '/kaggle/input/female-tfrecord-dataset/train_female.tfrecord-04-of-20',
 '/kaggle/input/male-tfrecord-dataset/train_male.tfrecord-14-of-20',
 '/kaggle/input/female-tfrecord-dataset/train_female.tfrecord-08-of-20',
 '/kaggle/input/male-tfrecord-dataset/train_male.tfrecord-17-of-20',
 '/kaggle/input/female-tfrecord-dataset/train_female.tfrecord-09-of-20',
 '/kag

### ${Creating\ dataset}$

In [5]:
train_set = preprocess.gender_dataset(
    filepaths=train_file_dataset,
    repeat=False,
    normalise=True,
    augment_image=None,
    shuffle_buffer_size=10_000,
    cache=None,
    batch_size=128,
    image_shape=(218, 178, 3),
    n_reads=tf.data.AUTOTUNE,
    prefetch=tf.data.AUTOTUNE,
)

valid_set = preprocess.gender_dataset(
    filepaths=valid_file_dataset,
    batch_size=128,
    image_shape=(218, 178, 3),
    n_reads=tf.data.AUTOTUNE,
    prefetch=tf.data.AUTOTUNE,
)

## Creating Model

In [6]:
model = create_model()

model.summary()

Model: "Gender_clf_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation_layer              │ (None, 218, 178, 3)    │             0 │
│ (AugmentationLayer)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov2d_1 (Conv2D)                │ (None, 218, 178, 32)   │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_1                     │ (None, 218, 178, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_1 (Activation) │ (None, 218, 178, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2d_1 (MaxPooling2D)      │ (None, 109, 89, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov2d_2 (Conv2D)                │ (None, 109, 89, 64)    │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_2                     │ (None, 109, 89, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_2 (Activation) │ (None, 109, 89, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2d_2 (MaxPooling2D)      │ (None, 54, 44, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov2d_3 (Conv2D)                │ (None, 54, 44, 96)     │        55,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_3                     │ (None, 54, 44, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_3 (Activation) │ (None, 54, 44, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2d_3 (MaxPooling2D)      │ (None, 27, 22, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov2d_4 (Conv2D)                │ (None, 27, 22, 128)    │       110,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_4                     │ (None, 27, 22, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_4 (Activation) │ (None, 27, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2d_4 (MaxPooling2D)      │ (None, 13, 11, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_layer_1 (Flatten)       │ (None, 18304)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer_1 (Dropout)       │ (None, 18304)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer_1 (Dense)           │ (None, 512)            │     9,371,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_5                     │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_5 (Activation) │ (None, 512)            │             

 Total params: 9,562,209 (36.48 MB)

 Trainable params: 9,560,545 (36.47 MB)

 Non-trainable params: 1,664 (6.50 KB)

## Model Weights Loading

Note that this cell would only be useful after the model has been trained in the next cell below for at least 1 epoch. Else one of the handled errors would likely be displayed but execution would not stop.

In [7]:
model_weights_filepath = '/kaggle/working/model_best_weights.weights.h5'

params = {
    'loss': tf.keras.losses.BinaryCrossentropy(from_logits=False),
    'optimizer': tf.keras.optimizers.Nadam(learning_rate=0.001),
    'metrics': ['accuracy']
}

try:
    model, eval_result = init_weights(
        model,
        dataset=valid_set, 
        params=params,
        retrieve_weights_from=model_weights_filepath,
        return_evaluation=True
    )
except OSError as e:
    print(f'Failed to load weights from {model_weights_filepath}: {e}\n')
    eval_result = {'loss':0.0, 'accuracy':0.0}
except ValueError as e:
    print(f'Invalid weight file at {model_weights_filepath}: {e}\n')
    eval_result = {'loss':0.0, 'accuracy':0.0}
except KeyError as e:
    print(f'ncompatible weights for the current model architecture: {e}\n')
    eval_result = {'loss':0.0, 'accuracy':0.0}
    
print(f'Evaluation result: {eval_result}')

Failed to load weights from /kaggle/working/model_best_weights.weights.h5: [Errno 2] Unable to synchronously open file (unable to open file: name = '/kaggle/working/model_best_weights.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Evaluation result: {'loss': 0.0, 'accuracy': 0.0}


## Creating Callbacks

In [8]:
# Create callbacks
lr_performance_schedule_cb = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.0005,
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='max',
    restore_best_weights=True,
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_weights_filepath,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    save_freq='epoch',
    initial_value_threshold=eval_result['accuracy']
)

callbacks = {
    'ReduceLROnPlateau': lr_performance_schedule_cb, 
    'ModelCheckpoint': checkpoint_cb,
    'EarlyStopping': early_stopping_cb
}

## Model Training

In [9]:
model_filepath = '/kaggle/working/model_1.keras'

hystroy = train.train_model(
    model,
    X_train=train_set,
    valid_set=valid_set,
    epochs=0,
    params=params,
    callbacks=callbacks,
    with_aug_layer=False,
    aug_layer_name=True,
    insert_dropout=tf.keras.layers.Dropout(rate=0.3, name='dropout_layer_2'),
    save_model_to=None,
    show_model_summary=True,
    initial_epoch=0,
)

Training with with_aug_layer=False


Model: "final_gender_clf_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cov2d_1 (Conv2D)                │ (None, 218, 178, 32)   │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_1                     │ (None, 218, 178, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_1 (Activation) │ (None, 218, 178, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2d_1 (MaxPooling2D)      │ (None, 109, 89, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov2d_2 (Conv2D)                │ (None, 109, 89, 64)    │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_2                     │ (None, 109, 89, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_2 (Activation) │ (None, 109, 89, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2d_2 (MaxPooling2D)      │ (None, 54, 44, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov2d_3 (Conv2D)                │ (None, 54, 44, 96)     │        55,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_3                     │ (None, 54, 44, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_3 (Activation) │ (None, 54, 44, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2d_3 (MaxPooling2D)      │ (None, 27, 22, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov2d_4 (Conv2D)                │ (None, 27, 22, 128)    │       110,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_4                     │ (None, 27, 22, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_4 (Activation) │ (None, 27, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool2d_4 (MaxPooling2D)      │ (None, 13, 11, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_layer_1 (Flatten)       │ (None, 18304)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer_1 (Dropout)       │ (None, 18304)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer_1 (Dense)           │ (None, 512)            │     9,371,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batchnorm_5                     │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ swish_activation_5 (Activation) │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer_2 (Dropout)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 9,562,209 (36.48 MB)

 Trainable params: 9,560,545 (36.47 MB)

 Non-trainable params: 1,664 (6.50 KB)

## Model Saving

Optionally load model's weights to return to the best model.

In [10]:
try:
    model.load_weights(model_weights_filepath)
    model.save(model_filepath)
except Exception as e:
    print(e)

[Errno 2] Unable to synchronously open file (unable to open file: name = '/kaggle/working/model_best_weights.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
